In [1]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
import time
import statistics
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

2022-03-11 09:58:02.812635: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-11 09:58:02.812738: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def create_model(features,layers,activation):
    model = Sequential()
    model.add(Dense(features, input_dim=features, activation=activation))
    for i in range(layers):
        model.add(Dense(features, activation=activation))
    model.add(Dense(5, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam')
    return model
    
def train_model(train,test,model):
    train_X = train.drop(['normal','dos','u2r','r2l','probe'],axis=1)
    test_X = train.drop(['normal','dos','u2r','r2l','probe'],axis=1)
    train_Y = train.loc[:,['normal','dos','u2r','r2l','probe']]
    test_Y = train.loc[:,['normal','dos','u2r','r2l','probe']]
    old_score = 0
    test_score_acc=[]
    train_score_acc=[]
    test_score_auc=[]
    train_score_auc=[]
    model_time = []
    test_norm_mat=[]
    test_dos_mat=[]
    test_u2r_mat=[]
    test_r2l_mat=[]
    test_probe_mat=[]
    epoch = 0
    while (epoch < 2) or (test_score_auc[epoch-1] > test_score_auc[epoch-2]):
        model_time1 = time.perf_counter()
        model.fit(train_X,train_Y,epochs=1,batch_size=128)
        model_time2 = time.perf_counter()
        model_time.append(model_time2 - model_time1)
        yhat = model.predict(train_X)
        yhat = yhat.round()
        train_score_acc.append(accuracy_score(train_Y,yhat))
        train_score_auc.append(roc_auc_score(train_Y,yhat))
        yhat = model.predict(test_X)
        yhat = yhat.round()
        yhat1=[[] for _ in range(5)]
        for i in range(len(yhat)):
            for j in range(5):
                yhat1[j].append(yhat[i][j])
        test_score_acc.append(accuracy_score(test_Y,yhat))
        test_score_auc.append(roc_auc_score(test_Y,yhat))
        test_norm_mat=confusion_matrix(test_Y['normal'],yhat1[0])
        test_dos_mat=confusion_matrix(test_Y['dos'],yhat1[1])
        test_u2r_mat=confusion_matrix(test_Y['u2r'],yhat1[2])
        test_r2l_mat=confusion_matrix(test_Y['r2l'],yhat1[3])
        test_probe_mat=confusion_matrix(test_Y['probe'],yhat1[4])
        epoch = epoch + 1
    avg_time = statistics.mean(model_time[:-1])
    total_time=sum(model_time[:-1])
    return (test_score_acc[:-1], train_score_acc[:-1], test_score_auc[:-1], train_score_auc[:-1], avg_time, total_time, test_norm_mat[:-1], test_dos_mat[:-1], test_u2r_mat[:-1], test_r2l_mat[:-1], test_probe_mat[:-1])
    



In [3]:
test = pd.read_csv("../data/processed/test3.csv").drop(columns = ['Unnamed: 0'])
train = pd.read_csv("../data/processed/train3.csv").drop(columns = ['Unnamed: 0'])
features = train.shape[1] -5

In [4]:
model1_tanh = create_model(features,1,'tanh')
model1_relu = create_model(features,1,'relu')
model1_swish = create_model(features,1,'swish')
model3_tanh = create_model(features,3,'tanh')
model3_relu = create_model(features,3,'relu')
model3_swish = create_model(features,3,'swish')
model5_tanh = create_model(features,5,'tanh')
model5_relu = create_model(features,5,'relu')
model5_swish = create_model(features,5,'swish')
model10_tanh = create_model(features,10,'tanh')
model10_relu = create_model(features,10,'relu')
model10_swish = create_model(features,10,'swish')

2022-03-11 09:58:50.935139: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-11 09:58:50.935285: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-11 09:58:50.935365: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c-VirtualBox): /proc/driver/nvidia/version does not exist
2022-03-11 09:58:50.967943: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
score=[]
print(train_model(train,test,model1_tanh))
score.append(train_model(train,test,model1_tanh))
print('counter')
print(len(score))
score.append(train_model(train,test,model1_relu))
print('counter')
print(len(score))
score.append(train_model(train,test,model1_swish))
print('counter')
print(len(score))
score.append(train_model(train,test,model3_tanh))
print('counter')
print(len(score))
score.append(train_model(train,test,model3_relu))
print('counter')
print(len(score))
score.append(train_model(train,test,model3_swish))
print('counter')
print(len(score))
score.append(train_model(train,test,model5_tanh))
print('counter')
print(len(score))
score.append(train_model(train,test,model5_relu))
print('counter')
print(len(score))
score.append(train_model(train,test,model5_swish))
print('counter')
print(len(score))
score.append(train_model(train,test,model10_tanh))
print('counter')
print(len(score))
score.append(train_model(train,test,model10_relu))
print('counter')
print(len(score))
score.append(train_model(train,test,model10_swish))
print('counter')
print(len(score))
with open('../scores3','wb') as f:
	pickle.dump(score, f)

2895/2895 [==============================] - 9s 3ms/step - loss: 0.0016
([0.9983185566036463, 0.9988583458159589], [0.9983185566036463, 0.9988583458159589], [0.9247282945106889, 0.9403713165772125], [0.9247282945106889, 0.9403713165772125], 11.588707339499706, 23.177414678999412, array([[297464,    159]]), array([[76733,    87]]), array([[370474,      0]]), array([[369603,     74]]), array([[367432,     34]]))
2895/2895 [==============================] - 9s 3ms/step - loss: 0.0013


KeyboardInterrupt: 